##### Importing necessary modules for the project

In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import warnings
from sklearn import preprocessing
warnings.filterwarnings('ignore')
import time
import datetime
import scipy.stats
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics


In [ ]:
# Loading bids.csv

bids_file = pd.read_csv('/content/gdrive/MyDrive/CSV/Online Bid Dataset /bids.csv')

# Loading train.csv and test.csv
train = pd.read_csv('/content/gdrive/MyDrive/CSV/Online Bid Dataset /train.csv')

test = pd.read_csv('/content/gdrive/MyDrive/CSV/Online Bid Dataset /test.csv')

In [ ]:
# Joining the train data set with full data set

bids = pd.merge(train, bids_file, on='bidder_id', how='left')

# writing the merged dataset to file - bid.csv

bids.to_csv("/content/gdrive/MyDrive/CSV/Online Bid Dataset /bid.csv")

In [ ]:
# Loading merged csv file

bids_data = pd.read_csv("/content/gdrive/MyDrive/CSV/Online Bid Dataset /bid.csv")
bids_data.head()

,Unnamed: 0,bidder_id,payment_account_x,address_x,outcome_x,Unnamed: 0.1,payment_account_y,address_y,outcome_y,Unnamed: 0.1.1,...,address_y.1,outcome_y.1,bid_id,auction,merchandise,device,time,country,ip,url
0,0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,0,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,0,...,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,85371.0,yitr4,home goods,phone35,9.759489e+15,ke,82.34.177.248,vasstdc27m7nks3
1,1,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,1,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,1,...,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,274487.0,btpyy,home goods,phone1004,9.760249e+15,zm,202.130.71.167,vasstdc27m7nks3
2,2,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,2,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,2,...,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,527436.0,kj2ko,home goods,phone4,9.762261e+15,ke,22.54.76.225,vasstdc27m7nks3
3,3,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,3,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,3,...,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,554135.0,1m5t8,home goods,phone4,9.762431e+15,ke,32.13.237.126,vasstdc27m7nks3
4,4,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,4,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,4,...,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,607628.0,btpyy,home goods,phone4,9.762789e+15,ke,202.236.188.240,vasstdc27m7nks3


# Data Cleaning and Preprocessing

In [ ]:
pd.isnull(bids_data).any()

Unnamed: 0             False
bidder_id              False
payment_account_x      False
address_x              False
outcome_x              False
Unnamed: 0.1           False
payment_account_y      False
address_y              False
outcome_y              False
Unnamed: 0.1.1         False
payment_account_x.1    False
address_x.1            False
outcome_x.1            False
Unnamed: 0.1.1.1       False
payment_account_y.1    False
address_y.1            False
outcome_y.1            False
bid_id                  True
auction                 True
merchandise             True
device                  True
time                    True
country                 True
ip                      True
url                     True
dtype: bool

# Handling Missing Values

In [ ]:
# checking if there are any bidder id that will not have any bids

nobids_Train = bids_data[pd.isnull(bids_data['bid_id'])]['bidder_id'].unique()
len(nobids_Train)

29

In [ ]:
bids_data[bids_data['bidder_id'].isin(nobids_Train)]['outcome_x']

17583      0.0
42424      0.0
129439     0.0
214915     0.0
221776     0.0
226432     0.0
255107     0.0
305088     0.0
333331     0.0
409451     0.0
654549     0.0
657208     0.0
788183     0.0
790914     0.0
794927     0.0
810369     0.0
883351     0.0
1018748    0.0
1019599    0.0
1021349    0.0
1123023    0.0
1139921    0.0
1217466    0.0
1465813    0.0
1482542    0.0
1648201    0.0
1974826    0.0
2139077    0.0
2999380    0.0
Name: outcome_x, dtype: float64

In [ ]:
# For all missing values, dropping the observation

bids = bids_data[~bids_data['bidder_id'].isin(nobids_Train)]

In [ ]:
pd.isnull(bids).any()


Unnamed: 0             False
bidder_id              False
payment_account_x      False
address_x              False
outcome_x              False
Unnamed: 0.1           False
payment_account_y      False
address_y              False
outcome_y              False
Unnamed: 0.1.1         False
payment_account_x.1    False
address_x.1            False
outcome_x.1            False
Unnamed: 0.1.1.1       False
payment_account_y.1    False
address_y.1            False
outcome_y.1            False
bid_id                 False
auction                False
merchandise            False
device                 False
time                   False
country                 True
ip                     False
url                    False
dtype: bool

In [ ]:
bids = bids.dropna()

In [ ]:
# double checking the missing values

pd.isnull(bids).any()

Unnamed: 0             False
bidder_id              False
payment_account_x      False
address_x              False
outcome_x              False
Unnamed: 0.1           False
payment_account_y      False
address_y              False
outcome_y              False
Unnamed: 0.1.1         False
payment_account_x.1    False
address_x.1            False
outcome_x.1            False
Unnamed: 0.1.1.1       False
payment_account_y.1    False
address_y.1            False
outcome_y.1            False
bid_id                 False
auction                False
merchandise            False
device                 False
time                   False
country                False
ip                     False
url                    False
dtype: bool

# Exploratory data analysis

In [ ]:
# Ratio for human vs robot bids

print(
    f"total bids made by robots in data set: {len(bids[bids['outcome_x'] == 1])}")
print(
    f"total bids made by human in data set:  {len(bids[bids['outcome_y'] == 0])}")

total bids made by robots in data set: 412153
total bids made by human in data set:  2656370


In [ ]:
# Checking the unique number of each feature in the dataset

print(f"total row in bid:          {len(bids)}")
print(f"total bids in bid:         {len(bids['bid_id'].unique())}")
print(f"total bidder in bids:      {len(bids['bidder_id'].unique())}")
print(f"total payment in bids:     {len(bids['payment_account_x'].unique())}")
print(f"total address in bids:     {len(bids['address_x'].unique())}")
print(f"total auction in bids:     {len(bids['auction'].unique())}")
print(f"total merchandise in bids: {len(bids['merchandise'].unique())}")
print(f"total device in bids:      {len(bids['device'].unique())}")
print(f"total country in bids:     {len(bids['country'].unique())}")
print(f"total ip in bids:          {len(bids['ip'].unique())}")
print(f"total url in bids:         {len(bids['url'].unique())}")

total row in bid:          3068523
total bids in bid:         3068523
total bidder in bids:      1983
total payment in bids:     1983
total address in bids:     1983
total auction in bids:     12740
total merchandise in bids: 10
total device in bids:      5726
total country in bids:     198
total ip in bids:          1028810
total url in bids:         663265


# Feature Extraction

In [ ]:
def ent(bids_data):
    p_data = bids_data.value_counts()/len(bids_data)
    entropy_value = scipy.stats.entropy(p_data)
    return entropy_value

In [ ]:
# Bidding time difference - per user (bidder_id)
bids = bids.sort_values(by=['time'])
bids['timediffs'] = bids.groupby('bidder_id')['time'].transform(pd.Series.diff)

# Number of bids a user made in each auction
bids_PerAuction = bids.groupby(['auction', 'bidder_id']).size()
bids_PerAuction = bids_PerAuction.to_frame()

# Proportion of bots : for each Country
pbotsCountry = bids[bids['outcome_x'] == 1].groupby(
    'country').size()/bids.groupby('country').size()
pbotsCountry = pbotsCountry.fillna(0)
pbotsCountry = pbotsCountry.to_frame()

# Proportion of bots : for per Device
pbots_per_device = bids[bids['outcome_y'] == 1].groupby(
    'device').size()/bids.groupby('device').size()
pbots_per_device = pbots_per_device.fillna(0)
pbots_per_device = pbots_per_device.to_frame()

# Number of unique ip to : number of bids ratio
ipToBids_Ratio = bids.groupby('bidder_id')['ip'].nunique(
)/bids.groupby('bidder_id')['bid_id'].nunique()
ipToBids_Ratio = ipToBids_Ratio.to_frame()

# Mean for per auction url entropy for each user
# Input a pandas series
auctionUrl_entropy = bids.groupby(['auction', 'bidder_id'])['url'].apply(ent)
auctionUrl_entropy = auctionUrl_entropy.groupby(
    'bidder_id').mean().reset_index()

KeyError: ignored

In [ ]:
# merge the features back
bids = pd.merge(bids, bids_PerAuction, on=['auction', 'bidder_id'], how='left')
bids = pd.merge(bids, pbotsCountry, on='country', how='left')
bids = pd.merge(bids, pbots_per_device, on='device', how='left')
bids = pd.merge(bids, ipToBids_Ratio, on='bidder_id', how='left')
bids = pd.merge(bids, auctionUrl_entropy, on='bidder_id', how='left')
# set column names
bids.columns = ['Unnamed: 0', 'bidder_id', 'payment_account', 'address', 'outcome',
               'bid_id', 'auction', 'merchandise', 'device', 'time', 'country',
               'ip', 'url', 'timediffs', 'bids_PerAuction', 'pbotsCountry', 'pbots_per_device',
               'ipToBids_Ratio', 'auctionUrl_entropy']

ValueError: ignored

In [ ]:
bids = pd.concat([bids.iloc[:, 4], bids.iloc[:, -6:]], axis=1)
bids.to_csv("featured.csv")

In [ ]:
bids = pd.read_csv("featured.csv")
bids = bids.iloc[:, 1:8]

# Data Visualization

In [ ]:
# Plotting the data

bots = bids.loc[bids.outcome == 1]
humans = bids.loc[bids.outcome == 0]

fig, axes = plt.subplots(3, 2, figsize=(14, 12), sharex=False)
sns.distplot(bots['bids_PerAuction'], hist=False, kde=True,
             bins=int(180/5), color='darkblue',
             kde_kws={'linewidth': 1.5}, ax=axes[0, 0])
sns.distplot(humans['bids_PerAuction'], hist=False, kde=True,
             bins=int(180/5), color='darkred',
             kde_kws={'linewidth': 1.5}, ax=axes[0, 0])

sns.distplot(bots['pbotsCountry'], hist=False, kde=True,
             bins=int(180/5), color='darkblue',
             kde_kws={'linewidth': 1.5}, ax=axes[0, 1])
sns.distplot(humans['pbotsCountry'], hist=False, kde=True,
             bins=int(180/5), color='darkred',
             kde_kws={'linewidth': 1.5}, ax=axes[0, 1])

sns.distplot(bots['pbots_per_device'], hist=False, kde=True,
             bins=int(180/5), color='darkblue', label='bots',
             kde_kws={'linewidth': 1.5}, ax=axes[1, 0])
sns.distplot(humans['pbots_per_device'], hist=False, kde=True,
             bins=int(180/5), color='darkred', label='human',
             kde_kws={'linewidth': 1.5}, ax=axes[1, 0])

sns.distplot(bots['ipToBids_Ratio'], hist=False, kde=True,
             bins=int(180/5), color='darkblue',
             kde_kws={'linewidth': 1.5}, ax=axes[1, 1])
sns.distplot(humans['ipToBids_Ratio'], hist=False, kde=True,
             bins=int(180/5), color='darkred',
             kde_kws={'linewidth': 1.5}, ax=axes[1, 1])

sns.distplot(bots['auctionUrl_entropy'], hist=False, kde=True,
             bins=int(180/5), color='darkblue',
             kde_kws={'linewidth': 1.5}, ax=axes[2, 0])
sns.distplot(humans['auctionUrl_entropy'], hist=False, kde=True,
             bins=int(180/5), color='darkred',
             kde_kws={'linewidth': 1.5}, ax=axes[2, 0])

sns.distplot(bots['timediffs'], hist=False, kde=True,
             bins=int(180/5), color='darkblue',
             kde_kws={'linewidth': 1.5}, ax=axes[2, 1])
sns.distplot(humans['timediffs'], hist=False, kde=True,
             bins=int(180/5), color='darkred',
             kde_kws={'linewidth': 1.5}, ax=axes[2, 1])
plt.legend(['bots', 'human'])
plt.show()

# Splitiing into train and test set and handling under sampling

In [ ]:
bid_train, bid_test = train_test_split(bids, test_size=0.2)
bots_train = bid_train.loc[bid_train.outcome == 1]
human_train = bid_train.loc[bid_train.outcome == 0]
human_sample = human_train.sample(n=len(bots_train))
bid_train_balance = pd.concat([bots_train, human_sample])

label_encoder = preprocessing.LabelEncoder()

for i in list(bid_train_balance.columns):
    bid_train_balance[i] = label_encoder.fit_transform(bid_train_balance[i])
    bid_train_balance[i].unique()

for i in list(bid_test.columns):
    bid_test[i] = label_encoder.fit_transform(bid_test[i])
    bid_test[i].unique()


Y_Train = bid_train_balance['outcome']
X_Train = bid_train_balance.iloc[:, -5:]
Y_Test = bid_test['outcome']
X_Test = bid_test.iloc[:, -5:]

# Model Building

# 1.Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_Train,Y_Train)


lr_Predictions = lr.predict(X_Test)

print(metrics.f1_score(Y_Test, lr_Predictions, average='weighted'))

In [ ]:
lr_score = lr.predict_proba(X_Test)[:, 1]
fpr_lr, tpr_lr, _lr = roc_curve(Y_Test, lr_score)
roc_lr_auc = auc(fpr_lr, tpr_lr)

fig = plt.figure(figsize=(5, 5))
plt.plot(fpr_lr, tpr_lr, label='Logistic Regression : ROC curve (area = %0.2f)' % roc_lr_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.005])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()
fig.savefig('roc_lr_auc.png')

In [ ]:
print('Classification Report for Logistic Regression Model')
print(classification_report(Y_Test, lr_Predictions))

# 2.Random Forest

In [ ]:
randomForest = RandomForestClassifier(n_estimators=100, max_depth=5,
                            max_leaf_nodes=11, max_features='log2',
                            bootstrap=True, oob_score=True)
randomForest.fit(X_Train, Y_Train)

In [ ]:
rf_Predictions = randomForest.predict(X_Test)
print(f"Random Forest Accuracy: {accuracy_score(rf_Predictions, Y_Test):.3f}")

In [ ]:
y_RF_Score = randomForest.predict_proba(X_Test)[:, 1]
fpr_RF, tpr_RF, _RF = roc_curve(Y_Test, y_RF_Score)
roc_RF_auc = auc(fpr_RF, tpr_RF)

plt.figure(figsize=(5, 5))
plt.plot(fpr_RF, tpr_RF, label='Random Forest : ROC curve (area = %0.2f)' % roc_RF_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.005])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
print('Classification Report for Random Forest Model : ')
print(classification_report(Y_Test, rf_Predictions))

# 3.Gradient Boosting

In [ ]:
gb = GradientBoostingClassifier(n_estimators=10)
gb = GradientBoostingClassifier(n_estimators=100, max_depth=5, max_features='sqrt',
                                max_leaf_nodes=9)
gb.fit(X_Train, Y_Train)

In [ ]:
gb_Predictions = gb.predict(X_Test)
print(f" Accuracy for Gradient Boosting : {accuracy_score(gb_Predictions, Y_Test):.3f}")

In [ ]:
y_GB_score = gb.predict_proba(X_Test)[:, 1]
fpr_GB, tpr_GB, _GB = roc_curve(Y_Test, y_GB_score)
roc_GB_auc = auc(fpr_GB, tpr_GB)

plt.figure(figsize=(5, 5))
plt.plot(fpr_GB, tpr_GB, label='GB ROC curve : (area = %0.2f)' % roc_GB_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.005])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

In [ ]:
print('Classification Report for Gradient Boosting Model : ')
print(classification_report(Y_Test, gb_Predictions))

# Comparing Models

In [ ]:
lr_model = [metrics.accuracy_score(Y_Test, lr_Predictions), metrics.precision_score(Y_Test,lr_Predictions),
              metrics.recall_score(Y_Test, lr_Predictions), metrics.f1_score(Y_Test,lr_Predictions),
              metrics.roc_auc_score(Y_Test, lr_Predictions)]
rf_model = [metrics.accuracy_score(Y_Test, rf_Predictions), metrics.precision_score(Y_Test, rf_Predictions),
              metrics.recall_score(Y_Test, rf_Predictions), metrics.f1_score(
                  Y_Test, rf_Predictions),
              metrics.roc_auc_score(Y_Test, rf_Predictions)]
gb_model = [metrics.accuracy_score(Y_Test, gb_Predictions), metrics.precision_score(Y_Test, gb_Predictions),
              metrics.recall_score(Y_Test, gb_Predictions), metrics.f1_score(
                  Y_Test, gb_Predictions),
              metrics.roc_auc_score(Y_Test, gb_Predictions)]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
index = np.arange(5)
width = 0.2
bar1 = plt.bar(index, lr_model[0:5], width,
             alpha=0.4, color='orange', label='Logistic Regression')
bar2 = plt.bar(index+width, rf_model[0:5], width,
             alpha=0.8, color='green', label='random forest')
bar3 = plt.bar(index+2*width, gb_model[0:5], width,
             alpha=0.8, color='blue', label='gradient boosting')
plt.title('Model Comparison for the project :')
plt.ylabel('Score')
plt.xticks(index+width, ('accuracy', 'precision', 'recall', 'F1', 'ROC AUC'))
plt.legend(loc=8, ncol=3, mode="expand", borderaxespad=0.)
plt.show()

In [ ]:
label = ["Accuracy_score", "Precision_score",
         "Recall_Score", "F1_score", "ROC_AUC_score"]
table = pd.DataFrame({'Logistic Regression': lr_model,
                      'Random Forest': rf_model, 'Gradient Boosting': gb_model})
table = table.transpose()
table.columns = label
table.transpose().round(3)